# How to use a SmartLLMChain

A SmartLLMChain is a form of self-critique chain that can help you if have particularly complex questions to answer. Instead of doing a single LLM pass, it instead performs these 3 steps:
1. Ideation: Pass the user prompt n times through the LLM to get n output proposals (called "ideas"), where n is a parameter you can set 
2. Critique: The LLM critiques all ideas to find possible flaws and picks the best one 
3. Resolve: The LLM tries to improve upon the best idea (as chosen in the critique step) and outputs it. This is then the final output.

SmartLLMChains are based on the SmartGPT workflow proposed in https://youtu.be/wVzuvf9D9BU.

Note that SmartLLMChains
- use more LLM passes (ie n+2 instead of just 1)
- only work then the underlying LLM has the capability for reflection, whicher smaller models often don't
- only work with underlying models that return exactly 1 output, not multiple

This notebook demonstrates how to use a SmartLLMChain.

##### Same LLM for all steps

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import SmartLLMChain

We first create an LLM and prompt template

In [3]:
llm=OpenAI(temperature=0)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

Now we can create a SmartLLMChain

In [4]:
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)

Now we can use the SmartLLM as a drop-in replacement for our LLM. E.g.:

In [ ]:
chain.run("colorful socks")

##### Different LLM for different steps

You can also use different LLMs for the different steps by passing `ideation_llm`, `critique_llm` and `resolve_llm`. You might want to do this to use a more creative (i.e., high-temperature) model for ideation and a more strict (i.e., low-temperature) model for critique and resolution.

In [8]:
chain = SmartLLMChain(
    ideation_llm=OpenAI(temperature=0.9),
    llm=OpenAI(temperature=0),  # will be used for critqiue and resolution as no specific llms are given 
    prompt=prompt,
    n_ideas=3,
    verbose=True
)

#### Use with ChatModel 

SmartLLMs also work with ChatModels

In [5]:
from langchain.chat_models import ChatOpenAI

In [6]:
chain = SmartLLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt, verbose=True)